
# Teacher vs. Course Classification
Binary classifier to route feedback to the right workflow; includes heuristics, classical models, transformer probe, and cross-domain stress tests.


In [ ]:

import pandas as pd
from pathlib import Path

DATA_PATH = Path('data_feedback.xlsx')
df = pd.read_excel(DATA_PATH)
print(df.head())


In [ ]:

print('Label counts:')
print(df['teacher/course'].value_counts())



## 1. Keyword heuristic
Useful as a sanity check and for interpretable fallbacks.


In [ ]:

import re

def heuristic_label(text):
    text_l = text.lower()
    if re.search(r'teacher|sir|madam|teacher's', text_l):
        return 'teacher'
    if re.search(r'course|syllabus|curriculum', text_l):
        return 'course'
    return 'teacher'  # default toward teacher feedback

heuristic_preds = df['comments'].apply(heuristic_label)
from sklearn.metrics import accuracy_score
print('Heuristic accuracy:', accuracy_score(df['teacher/course'], heuristic_preds))



## 2. TF–IDF + Logistic Regression baseline


In [ ]:

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

X_train, X_val, y_train, y_val = train_test_split(
    df['comments'], df['teacher/course'], test_size=0.3, random_state=42, stratify=df['teacher/course']
)

word_lr = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1,2), min_df=1)),
    ('clf', LogisticRegression(max_iter=200, class_weight='balanced'))
])
word_lr.fit(X_train, y_train)
preds = word_lr.predict(X_val)
print(classification_report(y_val, preds))
ConfusionMatrixDisplay.from_predictions(y_val, preds, normalize='true', cmap='Blues')
plt.title('Teacher vs course baseline')
plt.show()



## 3. Character model for spelling robustness


In [ ]:

from sklearn.feature_extraction.text import TfidfVectorizer as CharTfidf

char_lr = Pipeline([
    ('tfidf', CharTfidf(analyzer='char', ngram_range=(3,5), min_df=1)),
    ('clf', LogisticRegression(max_iter=200, class_weight='balanced'))
])
char_lr.fit(X_train, y_train)
char_preds = char_lr.predict(X_val)
print(classification_report(y_val, char_preds))
ConfusionMatrixDisplay.from_predictions(y_val, char_preds, normalize='true', cmap='Reds')
plt.title('Character baseline')
plt.show()



## 4. Cross-task generalization stress test
Train on positive-sentiment subset, test on neutral/negative (or vice versa) to check robustness to sentiment shift.


In [ ]:

from sklearn.metrics import accuracy_score

pos_mask = df['sentiment'].str.lower().eq('positive')
nonpos_mask = ~pos_mask

model = word_lr
model.fit(df.loc[pos_mask, 'comments'], df.loc[pos_mask, 'teacher/course'])
pos_to_non = accuracy_score(df.loc[nonpos_mask, 'teacher/course'], model.predict(df.loc[nonpos_mask, 'comments']))

model.fit(df.loc[nonpos_mask, 'comments'], df.loc[nonpos_mask, 'teacher/course'])
non_to_pos = accuracy_score(df.loc[pos_mask, 'teacher/course'], model.predict(df.loc[pos_mask, 'comments']))
print({'train_pos_test_nonpos': pos_to_non, 'train_nonpos_test_pos': non_to_pos})



## 5. Transformer probe (zero-/few-shot)


In [ ]:

from transformers import pipeline

zs = pipeline('zero-shot-classification', model='facebook/bart-large-mnli')
labels = ['teacher','course']
example = df['comments'].iloc[2]
print(zs(example, candidate_labels=labels, hypothesis_template='This feedback is about the {label}.'))



## 6. Agentic router combining heuristic + model confidence


In [ ]:

probs = word_lr.predict_proba(X_val)
threshold = 0.6
combined_preds = []
for text, p in zip(X_val, probs):
    max_p = p.max()
    if max_p >= threshold:
        combined_preds.append(word_lr.classes_[p.argmax()])
    else:
        combined_preds.append(heuristic_label(text))

print('Hybrid accuracy:', accuracy_score(y_val, combined_preds))



## 7. Error analysis with aspect/context tags


In [ ]:

errors = pd.DataFrame({'text': X_val, 'true': y_val, 'pred': preds})
errors = errors[errors['true'] != errors['pred']]
errors['mentions_teacher'] = errors['text'].str.contains('teacher|sir|madam', case=False)
errors['mentions_course'] = errors['text'].str.contains('course|syllabus|curriculum', case=False)
print(errors)
